In [1]:
#import
import os

#basics
import pandas as pd
import numpy as np

#for timing code
from datetime import datetime, date


#plotting libraries
%matplotlib inline
import seaborn as sns
from seaborn import plt

# Some nice default configuration for plots
plt.rcParams['figure.figsize'] = 10, 7.5
plt.rcParams['axes.grid'] = True

#for AWS connection
from boto.s3.connection import S3Connection
from IPython.parallel import Client

#### Connect to AWS and pull in data:
See file: avs_data_script_auduno_v2_noFeatures

In [2]:
#get creds 
#credentials = pd.read_csv('/Users/rychughes/.ssh/credentials.csv') #local
credentials = pd.read_csv('/home/centos/certificates/credentials.csv') #AWS
aws_id = credentials['Access Key Id'][0]
aws_key = credentials['Secret Access Key'][0]

In [3]:
#connect to S3 for data
s3conn = S3Connection( aws_id , aws_key )
bucket = s3conn.get_bucket('avs-kaggle-data')

#### Load transaction data

In [4]:
keys = bucket.get_all_keys(prefix='reduced')
print len( keys)
#offers_keys
keys[0].get_contents_to_filename('reduced.csv.gz') #compressed file = 240mb, command hangs local machine... ~5 seconds on AWS

1


In [5]:
reduced_df = pd.read_csv('reduced.csv.gz',compression='gzip')
print reduced_df.shape
reduced_df.head(3)

(27764694, 11)


id  chain  dept  category    company  brand        date  productsize  \
0  86246    205    99      9909  104538848  15343  2012-03-02           16   
1  86246    205    21      2106  105100050  27873  2012-03-02           64   
2  86246    205    26      2630  103700030  14647  2012-03-02           56   

  productmeasure  purchasequantity  purchaseamount  
0             OZ                 1            2.49  
1             OZ                 1            3.29  
2             CT                 1            1.00

#### Load offers data

In [6]:
keys = bucket.get_all_keys(prefix='offers')
print len( keys)
keys[0].get_contents_to_filename('offers.csv')

1


In [7]:
offers_df = pd.read_csv('offers.csv')
print offers_df.shape
offers_df.head(3)

(37, 6)


offer  category  quantity    company  offervalue  brand
0  1190530      9115         1  108500080        5.00  93904
1  1194044      9909         1  107127979        1.00   6732
2  1197502      3203         1  106414464        0.75  13474

#### load train history data

In [8]:
keys = bucket.get_all_keys(prefix='trainHistory')
print len( keys)
keys[0].get_contents_to_filename('trainHistory.csv')

1


In [9]:
X_train = pd.read_csv('trainHistory.csv')
print X_train.shape
X_train.head(3)

(160057, 7)


id  chain    offer  market  repeattrips repeater   offerdate
0     86246    205  1208251      34            5        t  2013-04-24
1     86252    205  1197502      34           16        t  2013-03-27
2  12682470     18  1197502      11            0        f  2013-03-28

#### load test history data

In [10]:
keys = bucket.get_all_keys(prefix='testHistory')
print len( keys)
keys[0].get_contents_to_filename('testHistory.csv')

1


In [11]:
X_test = pd.read_csv('testHistory.csv')
print X_test.shape
X_test.head(3)

(151484, 5)


id  chain    offer  market   offerdate
0  12262064     95  1219903      39  2013-06-27
1  12277270     95  1221658      39  2013-06-23
2  12332190     95  1213242      39  2013-06-15

#### Helpful functions for exploring data

In [12]:
#get unique value counts for each field
def print_unique_value_counts_per_attribute( df ):
    #get col headers
    col_headers = list( df.columns)

    for i in range( 0, len(col_headers), 1):
        print( col_headers[i], str(len(pd.value_counts( df[ col_headers[i] ]))) )

#### merge dataset
transaction merge with history by (id, chain), transaction merge with offer by (category, brand, company)

In [13]:
#combine X_train and X_test for feature creation
X_combine = pd.concat([X_train, X_test])
print X_combine.shape
X_combine.head(1) #view train sample

(311541, 7)


chain     id  market    offer   offerdate repeater  repeattrips
0    205  86246      34  1208251  2013-04-24        t            5

In [14]:
print X_combine.shape
X_combine.tail(1) #view test sample

(311541, 7)


chain          id  market    offer   offerdate repeater  repeattrips
151483     46  4853598737      15  1230218  2013-07-27      NaN          NaN

In [15]:
X_combine = pd.merge( X_combine, offers_df, how='left', on=['offer'])
print X_combine.shape
X_combine.head(3) 

(311541, 12)


chain        id  market    offer   offerdate repeater  repeattrips  \
0    205     86246      34  1208251  2013-04-24        t            5   
1    205     86252      34  1197502  2013-03-27        t           16   
2     18  12682470      11  1197502  2013-03-28        f            0   

   category  quantity    company  offervalue  brand  
0      2202         1  104460040        2.00   3718  
1      3203         1  106414464        0.75  13474  
2      3203         1  106414464        0.75  13474

In [16]:
X_combine.info()
#repeater & repeattrips include non-null values, as expected

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311541 entries, 0 to 311540
Data columns (total 12 columns):
chain          311541 non-null int64
id             311541 non-null int64
market         311541 non-null int64
offer          311541 non-null int64
offerdate      311541 non-null object
repeater       160057 non-null object
repeattrips    160057 non-null float64
category       311541 non-null int64
quantity       311541 non-null int64
company        311541 non-null int64
offervalue     311541 non-null float64
brand          311541 non-null int64
dtypes: float64(2), int64(8), object(2)
memory usage: 30.9+ MB


In [17]:
print_unique_value_counts_per_attribute( X_combine)

('chain', '134')
('id', '311541')
('market', '34')
('offer', '37')
('offerdate', '145')
('repeater', '2')
('repeattrips', '56')
('category', '20')
('quantity', '2')
('company', '18')
('offervalue', '7')
('brand', '19')


#### Explore transaction dataset

In [18]:
#confirm dataset is complete... does NOT provide "count", but does show that memory usage is 2.5+ GB 
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27764694 entries, 0 to 27764693
Data columns (total 11 columns):
id                  int64
chain               int64
dept                int64
category            int64
company             int64
brand               int64
date                object
productsize         float64
productmeasure      object
purchasequantity    int64
purchaseamount      float64
dtypes: float64(2), int64(7), object(2)
memory usage: 2.5+ GB


In [19]:
reduced_df.describe().T

count          mean           std         min  \
id                27764694  1.846040e+09  1.547659e+09    86246.00   
chain             27764694  1.281176e+02  1.255024e+02        2.00   
dept              27764694  4.725891e+01  2.719405e+01        2.00   
category          27764694  4.739659e+03  2.718690e+03      201.00   
company           27764694  1.838715e+08  2.660773e+08  1036030.00   
brand             27764694  1.273590e+04  1.497814e+04        0.00   
productsize       27764694  2.873390e+01  4.233549e+01        0.00   
purchasequantity  27764694  1.490484e+00  3.886984e+00      -27.00   
purchaseamount    27764694  5.286882e+00  9.019249e+00     -885.04   

                          25%           50%           75%           max  
id                497991065.0  7.658742e+08  3.343457e+09  4.853599e+09  
chain                    26.0  9.600000e+01  1.650000e+02  5.260000e+02  
dept                     26.0  3.700000e+01  5.800000e+01  9.900000e+01  
category               2628.0  3.703000e+03  5.824000e+03  9.909000e+03  
company           103700030.0  1.046100e+08  1.071280e+08  1.088882e+10  
brand                  3342.0  8.993000e+03  1.619600e+04  1.086130e+05  
productsize               8.0  1.450000e+01  3.200000e+01  1.080000e+03  
purchasequantity          1.0  1.000000e+00  2.000000e+00  1.184500e+04  
purchaseamount            2.5  3.790000e+00  5.990000e+00  2.072875e+04

In [20]:
print_unique_value_counts_per_attribute( reduced_df )

('id', '311201')
('chain', '134')
('dept', '54')
('category', '209')
('company', '1572')
('brand', '2031')
('date', '514')
('productsize', '812')
('productmeasure', '6')
('purchasequantity', '592')
('purchaseamount', '21349')


#### Feature extraction
https://chitcode.wordpress.com/

In [24]:
def extract_features( cust_id):
    cust_details = X_combine.ix[X_combine.id == cust_id,:] #gets 12-attributes for customer (1x12 list)
    #print cust_details
    
    #assign important variables for feature creation
    offer_category = cust_details.category.values[0]
    offer_company = cust_details.company.values[0]
    offer_brand = cust_details.brand.values[0]
    #print offer_category, offer_company , offer_brand 
    
    #get all transactions for customer
    trans_cust = reduced_df.ix[reduced_df.id == cust_id,:]
    #print trans_cust
    
    #define empty transaction to catch exceptions in feature creation
    empty_trans = pd.DataFrame({'id':0,'chain':0,'dept':0,'category':0,'company':0,'brand':0,'date':'2000-01-01',
                                'productsize':0,'productmeasure':'OO','purchasequantity':0,'purchaseamount':0.0},index=[1])  
    if trans_cust.shape[0] == 0:        
        trans_cust=empty_trans
    
    #Chitcode features - https://chitcode.wordpress.com/
    #category
    cust_details['category_total_amount'] =   trans_cust.ix[trans_cust.category == offer_category,:].purchaseamount.sum()  
    cust_details['category_total_quantity'] = trans_cust.ix[trans_cust.category == offer_category,:].purchasequantity.sum()  
    cust_details['category_buy_true'] =  1 if trans_cust.ix[trans_cust.category == offer_category,:].purchasequantity.sum() > 0 else 0
    #company
    cust_details['company_total_amount'] =   trans_cust.ix[trans_cust.company == offer_company,:].purchaseamount.sum()  
    cust_details['company_total_quantity'] = trans_cust.ix[trans_cust.company == offer_company,:].purchasequantity.sum()  
    cust_details['company_buy_true'] =  1 if trans_cust.ix[trans_cust.company == offer_company,:].purchasequantity.sum() > 0 else 0
    #brand
    cust_details['brand_total_amount'] =   trans_cust.ix[trans_cust.brand == offer_brand,:].purchaseamount.sum()  
    cust_details['brand_total_quantity'] = trans_cust.ix[trans_cust.brand == offer_brand,:].purchasequantity.sum()  
    cust_details['brand_buy_true'] =  1 if trans_cust.ix[trans_cust.brand == offer_brand,:].purchasequantity.sum() > 0 else 0
    #print cust_details
    
    #Selected ML-Wave features go one layer deeper
    #TBD
    #print cust_details

    return cust_details


In [26]:
extract_features( 86246 ) #test


/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas

chain     id  market    offer   offerdate repeater  repeattrips  category  \
0    205  86246      34  1208251  2013-04-24        t            5      2202   

   quantity    company       ...        brand  category_total_amount  \
0         1  104460040       ...         3718                      0   

   category_total_quantity  category_buy_true  company_total_amount  \
0                        0                  0                243.63   

   company_total_quantity  company_buy_true  brand_total_amount  \
0                      37                 1               28.71   

   brand_total_quantity  brand_buy_true  
0                     9               1  

[1 rows x 21 columns]

In [25]:
'''
extract_features( 86246 ) #test

Warning for discussion with TAs: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
'''

'\nextract_features( 86246 ) #test\n\nWarning for discussion with TAs: \nA value is trying to be set on a copy of a slice from a DataFrame.\nTry using .loc[row_indexer,col_indexer] = value instead\n'

In [23]:
len(X_combine.id.values)

311541

In [25]:
X_features = pd.DataFrame()
X_features = pd.concat([X_features,extract_features(86246)]) #creating new df w/ features for each X_combine
print X_features.shape
X_features.head()

/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas

(1, 18)


chain     id  market    offer   offerdate repeater  repeattrips  category  \
0    205  86246      34  1208251  2013-04-24        t            5      2202   

   quantity    company  offervalue  brand  category_total_amount  \
0         1  104460040           2   3718                      0   

   category_total_quantity  company_total_amount  company_total_quantity  \
0                        0                243.63                      37   

   brand_total_amount  brand_total_quantity  
0               28.71                     9

In [ ]:

start_time = datetime.now() #for timing code

X_features = pd.DataFrame()

#create 
for e, cust_id in enumerate( X_combine.id.values): #test
#for e, cust_id in enumerate( X_combine.id.values): #complete
    try: 
        X_features = pd.concat([X_features,extract_features(cust_id)]) #creating new df w/ features for each X_combine
        #progress, spits out time taken for each 1000 cust_ids (311 total)
        if e%1000 == 0: 
            print e, dateTime.now()-start_time
    except:
        print '====='*10 #prints series of dashes to make errors "pop"
        print 'Error at id', cust_id

print X_features.shape
X_features.head(3)

/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/centos/miniconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas

In [38]:
X_features.to_csv('X_features.csv',index=False)
X_features = X_features.fillna(value=-1)